# **Amazon Lookout for Equipment** - Anomymized expander データセットのデモ
*Part 2: データセットの作成*

**バケットネームの変更:** このノートブックはS3バケットおよびLookout for EquipmentサービスにアクセスするためのIAMロールが必要です。この段階では、Lookout for EquipmentへデータをインジェストするためにS3上のファイルへの読み込み権限が必要です。後段のノートブックで推論スケジュールフェイズではバケットへの書き込み権限が必要になります。


**Note:** もし、Amazon Lookout for EquipmentのIAMロールを作っていない場合、次のインストラクションを参考にしてください。 [**set of instructions to create an IAM role**](https://github.com/dast1/l4e_iam_role_configuration/blob/main/configure_IAM_role.md).

In [2]:
BUCKET = 'lookout-equipment-ap-northeast-1-820974724107'
PREFIX = 'demo-data/training-data/expander/'

## Initialization
---
Following the data preparation notebook, this repository should now be structured as follow:
```
/lookout-equipment-demo
|
+-- data/
|   |
|   +-- labelled-data/
|   |   \-- labels.csv
|   |
|   \-- training-data/
|       \-- expander/
|           |-- subsystem-01
|           |   \-- subsystem-01.csv
|           |
|           |-- subsystem-02
|           |   \-- subsystem-02.csv
|           |
|           |-- ...
|           |
|           \-- subsystem-24
|               \-- subsystem-24.csv
|
+-- dataset/
|   |-- labels.csv
|   |-- tags_description.csv
|   |-- timeranges.txt
|   \-- timeseries.zip
|
+-- notebooks/
|   |-- 1_data_preparation.ipynb
|   |-- 2_dataset_creation.ipynb            <<< This notebook <<<
|   |-- 3_model_training.ipynb
|   |-- 4_model_evaluation.ipynb
|   \-- 5_inference_scheduling.ipynb
|
+-- utils/
    |-- lookout_equipment_utils.py
    \-- lookoutequipment.json
```

### ノートブックの設定のアップデート
Amazon Lookout for equipmentは最近登場したサービスであるため、AWS Pythonパッケージが最新版であることを確認する必要があります。もし`pip`依存関係のエラーが出た場合、`boto3`のバージョンを確認してください。1.17.48 以降のバージョンであれば(lookoutequipment APIを含むので問題ありませんので、このエラーを無視して先に進んで構いません。


In [4]:
!pip install --quiet --upgrade boto3 tqdm sagemaker

import boto3
print(f'boto3 version: {boto3.__version__} (should be >= 1.17.48 to include Lookout for Equipment API)')

# Restart the current notebook to ensure we take into account the previous updates:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

boto3 version: 1.17.74 (should be >= 1.17.48 to include Lookout for Equipment API)


### ライブラリのインポート

In [6]:
import boto3
import os
import pandas as pd
import pprint
import sagemaker
import sys
import time
import warnings

from datetime import datetime

# Helper functions for managing Lookout for Equipment API calls:
sys.path.append('../utils')
import lookout_equipment_utils as lookout

### パラメータ

In [8]:
warnings.filterwarnings('ignore')

DATA       = os.path.join('..', 'data')
LABEL_DATA = os.path.join(DATA, 'labelled-data')
TRAIN_DATA = os.path.join(DATA, 'training-data', 'expander')

ROLE_ARN = sagemaker.get_execution_role()
REGION_NAME = boto3.session.Session().region_name
DATASET_NAME = 'lookout-demo-training-dataset'   # Lookout for Equipmentで作成されるデータセット名

In [10]:
# List of the directories from the training data 
# directory: each directory corresponds to a subsystem:
components = []
for root, dirs, files in os.walk(f'{TRAIN_DATA}'):
    for subsystem in dirs:
        components.append(subsystem)

## データセットの作成
---

### データスキーマの作成

はじめに、データセットのスキーマをセットアップします。下のセルに示すように、`DATASET_COMPONENT_FIELDS_MAP`を定義してください。`DATASET_COMPONENT_FIELDS_MAP`はPythonの辞書形式のオブジェクトです。辞書中のキーは`Component`名、値は`Component`のカラム名のリストです。カラム名はcsvファイル中のヘッダと厳密に一致する必要があります。また、カラム名はcsvファイル中のヘッダのカラム名の順番と厳密に一致する必要があります。


```json
DATASET_COMPONENT_FIELDS_MAP = {
    "Component1": ['Timestamp', 'Tag1', 'Tag2',...],
    "Component2": ['Timestamp', 'Tag1', 'Tag2',...]
    ...
    "ComponentN": ['Timestamp', 'Tag1', 'Tag2',...]
}
```

今回のデータセットの例で書き直してみましょう。`DATASET_COMPONENT_FIELDS_MAP`は以下のようになります。
subsystem-01と呼ばれるコンポーネントが24個あり、それぞれのコンポーネントには、タイムスタンプ情報とタグ（センサシグナル）値が格納されます。各コンポーネント（subsystem）内のタグの数は同じである必要はありません。（ポンプ1に温度センサ5個、ポンプ2に温度センサ3個のように異なることはよくあります）。

コンポーネント名（この場合"subsystem-\*"）がS3中のフォルダー名と**厳密に一致する**ことを確認してください(大文字と小文字を区別します)。



```json
DATASET_COMPONENT_FIELDS_MAP = {
    "subsystem-01": ['Timestamp', 'signal-026', 'signal-027',... , 'signal-092'],
    "subsystem-02": ['Timestamp', 'signal-022', 'signal-023',... , 'signal-096'],
    ...
    "subsystem-24": ['Timestamp', 'signal-083'],
}
```

In [11]:
DATASET_COMPONENT_FIELDS_MAP = dict()
for subsystem in components:
    subsystem_tags = ['Timestamp']
    for root, _, files in os.walk(f'{TRAIN_DATA}/{subsystem}'):
        for file in files:
            fname = os.path.join(root, file)
            current_subsystem_df = pd.read_csv(fname, nrows=1)
            subsystem_tags = subsystem_tags + current_subsystem_df.columns.tolist()[1:]

        DATASET_COMPONENT_FIELDS_MAP.update({subsystem: subsystem_tags})
        
        
lookout_dataset = lookout.LookoutEquipmentDataset(
    dataset_name=DATASET_NAME,
    component_fields_map=DATASET_COMPONENT_FIELDS_MAP,
    region_name=REGION_NAME,
    access_role_arn=ROLE_ARN
)

もし、コンソールを使いたい場合、下記のpprintで表示した文字列を用いて、**データセットスキーマ**を設定することができます。
コンソール上でのデータセットの作成時に、下記に表示される文字列をData schemaの設定画面にペーストすることができます。


![dataset_schema](../assets/dataset-schema.png)

In [16]:
import pprint
pp = pprint.PrettyPrinter(depth=5)
pp.pprint(eval(lookout_dataset.dataset_schema))

{'Components': [{'Columns': [{'Name': 'Timestamp', 'Type': 'DATETIME'},
                             {'Name': 'signal-042', 'Type': 'DOUBLE'},
                             {'Name': 'signal-043', 'Type': 'DOUBLE'},
                             {'Name': 'signal-051', 'Type': 'DOUBLE'},
                             {'Name': 'signal-052', 'Type': 'DOUBLE'},
                             {'Name': 'signal-053', 'Type': 'DOUBLE'},
                             {'Name': 'signal-054', 'Type': 'DOUBLE'},
                             {'Name': 'signal-100', 'Type': 'DOUBLE'},
                             {'Name': 'signal-101', 'Type': 'DOUBLE'},
                             {'Name': 'signal-102', 'Type': 'DOUBLE'}],
                 'ComponentName': 'subsystem-11'},
                {'Columns': [{'Name': 'Timestamp', 'Type': 'DATETIME'},
                             {'Name': 'signal-001', 'Type': 'DOUBLE'},
                             {'Name': 'signal-002', 'Type': 'DOUBLE'},
                       

### データセットの作成

In [17]:
lookout_dataset.create()

EndpointConnectionError: Could not connect to the endpoint URL: "https://lookoutequipment.ap-northeast-1.amazonaws.com/"

The dataset is now created, but it is empty and ready to receive some timeseries data that we will ingest from the S3 location prepared in the previous notebook:

![dataset_schema](../assets/dataset-created.png)

## Ingest data into a dataset
---
Let's double check the values of all the parameters that will be used to ingest some data into an existing Lookout for Equipment dataset:

In [9]:
ROLE_ARN, BUCKET, PREFIX, DATASET_NAME

('arn:aws:iam::123031033346:role/service-role/AmazonSageMaker-ExecutionRole-20210128T070865',
 'sagemaker-lookout-equipment-demo',
 'data4/training-data/expander/',
 'lookout-demo-training-dataset-v4')

Launch the ingestion job in the Lookout for Equipment dataset:

In [10]:
response = lookout_dataset.ingest_data(BUCKET, PREFIX)

The ingestion is launched. With this amount of data (around 1.5 GB), it should take between 5-10 minutes:

![dataset_schema](../assets/dataset-ingestion-in-progress.png)

In [11]:
# Get the ingestion job ID and status:
data_ingestion_job_id = response['JobId']
data_ingestion_status = response['Status']

# Wait until ingestion completes:
print("=====Polling Data Ingestion Status=====\n")
lookout_client = lookout.get_client(region_name=REGION_NAME)
print(str(pd.to_datetime(datetime.now()))[:19], "| ", data_ingestion_status)

while data_ingestion_status == 'IN_PROGRESS':
    time.sleep(60)
    describe_data_ingestion_job_response = lookout_client.describe_data_ingestion_job(JobId=data_ingestion_job_id)
    data_ingestion_status = describe_data_ingestion_job_response['Status']
    print(str(pd.to_datetime(datetime.now()))[:19], "| ", data_ingestion_status)
    
print("\n=====End of Polling Data Ingestion Status=====")

=====Polling Data Ingestion Status=====

2021-04-16 20:03:50 |  IN_PROGRESS
2021-04-16 20:04:50 |  IN_PROGRESS
2021-04-16 20:05:50 |  IN_PROGRESS
2021-04-16 20:06:50 |  IN_PROGRESS
2021-04-16 20:07:50 |  IN_PROGRESS
2021-04-16 20:08:50 |  IN_PROGRESS
2021-04-16 20:09:50 |  IN_PROGRESS
2021-04-16 20:10:50 |  IN_PROGRESS
2021-04-16 20:11:50 |  SUCCESS

=====End of Polling Data Ingestion Status=====


The ingestion should now be complete as can be seen in the console:

![dataset_schema](../assets/dataset-ingestion-done.png)

## Conclusion
---

In this notebook, we created a **Lookout for Equipment dataset** and ingested the S3 data previously uploaded into this dataset. **Move now to the next notebook to train a model based on these data.**

In [12]:
# We'll just persist this dataset name to collect it from the next notebook in this series:
dataset_fname = os.path.join(DATA, 'dataset_name.txt')
with open(dataset_fname, 'w') as f:
    f.write(DATASET_NAME)